1. Data

1.1 Load

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

X = pd.read_csv('fetal_health.csv', sep = ';')

X.drop(X.tail(1900).index, inplace = True)

X.info()
y = X['fetal_health']
y = y.astype(int)
X = X.drop(columns = ['fetal_health'])
classes = set(y)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X = pd.DataFrame(X)

classes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 225
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          226 non-null    float64
 1   accelerations                                           226 non-null    float64
 2   fetal_movement                                          226 non-null    float64
 3   uterine_contractions                                    226 non-null    float64
 4   light_decelerations                                     226 non-null    float64
 5   severe_decelerations                                    226 non-null    float64
 6   prolongued_decelerations                                226 non-null    float64
 7   abnormal_short_term_variability                         226 non-null    float64
 8   mean_value_of_short_term_variability    

{1, 2, 3}

1.2 Bootstrap

In [2]:
print(X)
X['y_true'] = y
bts = []
for i in range(3):
  df_bt = X.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])


           0         1         2         3    4    5    6         7   \
0    0.133333  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.802817   
1    0.400000  0.352941  0.000000  0.461538  0.2  0.0  0.0  0.014085   
2    0.422222  0.176471  0.000000  0.615385  0.2  0.0  0.0  0.000000   
3    0.444444  0.176471  0.000000  0.615385  0.2  0.0  0.0  0.000000   
4    0.400000  0.411765  0.000000  0.615385  0.0  0.0  0.0  0.000000   
..        ...       ...       ...       ...  ...  ...  ...       ...   
221  0.333333  0.000000  0.038380  0.000000  0.0  0.0  0.0  0.549296   
222  0.333333  0.117647  0.027719  0.000000  0.0  0.0  0.0  0.436620   
223  0.333333  0.352941  0.036247  0.000000  0.0  0.0  0.0  0.394366   
224  0.333333  0.117647  0.034115  0.000000  0.0  0.0  0.0  0.521127   
225  0.333333  0.000000  0.040512  0.000000  0.0  0.0  0.0  0.563380   

           8         9   ...        11        12        13        14   15  \
0    0.000007  0.544304  ...  0.385714  0.110092  0.000000

1.3 Classification

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

models = [DecisionTreeClassifier(), LogisticRegression(), LinearSVC()]
preds= []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index = X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))

1.0
0.8230088495575221
0.8761061946902655


1.4 Aggregation of predictions

In [4]:
pred_df = pd.concat(preds, axis=1)
pred_df.head()
pred_df.to_csv('predicts.csv')

2. Ontology

In [5]:
!pip install owlready2
from owlready2 import *

onto = get_ontology('onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.5 MB 81.9 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.39-cp38-cp38-linux_x86_64.whl size=22260883 sha256=3894e191bfd21a2456fe1398c6fe4cd9e9a0aeb77c74c23becef3c940cb85699
  Stored in directory: /root/.cache/pip/wheels/bd/16/5f/a0bfc34a8f7682cbd6b4d9cb1436c0a0a04ac3579394d7e28a
Successfully built owlready2


2.x

In [6]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')

In [7]:
with onto:
  for index, row in pred_df.iterrows():
    inst_old = None
    for j, pred in enumerate(pred_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto.owl', format='ntriples')

In [8]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug = 2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/py

* Owlready * Adding relation onto.inst43_2 tsr onto.inst43_2
* Owlready * Adding relation onto.inst134_1 tsr onto.inst134_1
* Owlready * Adding relation onto.inst134_1 ts onto.inst134_1
* Owlready * Adding relation onto.inst73_1 tsr onto.inst73_1
* Owlready * Adding relation onto.inst73_1 ts onto.inst73_1
* Owlready * Adding relation onto.inst30_1 tsr onto.inst30_1
* Owlready * Adding relation onto.inst30_1 ts onto.inst30_1
* Owlready * Adding relation onto.inst26_1 tsr onto.inst26_1
* Owlready * Adding relation onto.inst106_1 tsr onto.inst106_1
* Owlready * Adding relation onto.inst106_1 ts onto.inst106_1
* Owlready * Adding relation onto.inst6_2 tsr onto.inst6_2
* Owlready * Adding relation onto.inst28_0 tsr onto.inst28_0
* Owlready * Adding relation onto.inst28_0 ts onto.inst28_0
* Owlready * Adding relation onto.inst98_0 tsr onto.inst98_0
* Owlready * Adding relation onto.inst98_0 ts onto.inst98_0
* Owlready * Adding relation onto.inst70_0 tsr onto.inst70_0
* Owlready * Adding rela

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [9]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

onto.save('onto_reasoned.owl', format='ntriples')

In [10]:
with onto:
  sync_reasoner_pellet(debug = 2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/py